In [9]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!git clone https://github.com/ItWasAllYellow/public_cs224n_gpt.git
%cd public_cs224n_gpt

Cloning into 'public_cs224n_gpt'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 69 (delta 21), reused 13 (delta 13), pack-reused 32 (from 1)
Receiving objects: 100% (69/69), 30.87 MiB | 25.66 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/public_cs224n_gpt/public_cs224n_gpt


In [11]:
"""
Answer Margin Ranking Trainer

Trains a GPT-2 based model to rank multiple-choice answers by minimizing margin loss
on perplexity scores. Generates Top-1/2/3 accuracy and average perplexity on dev/test splits.
"""

import argparse
import os
import random
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model      # public_cs224n_gpt implementation
from transformers import GPT2LMHeadModel

# suppress TQDM if needed
TQDM_DISABLE = False

def seed_everything(seed: int = 42):
    """Fix random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class AnswerMarginModel(nn.Module):
    """Wrapper around GPT2Model + LM head for computing token logits."""
    def __init__(self, args):
        super().__init__()
        # backbone
        self.gpt = GPT2Model.from_pretrained(
            model=args.model_size,
            d=args.d,
            l=args.l,
            num_heads=args.num_heads
        )
        # LM head
        self.lm_head = nn.Linear(args.d, GPT2Tokenizer.from_pretrained(args.model_size).vocab_size, bias=False)
        # fine-tune everything
        for p in self.gpt.parameters():
            p.requires_grad = True
        for p in self.lm_head.parameters():
            p.requires_grad = True

    def forward(self, input_ids, attention_mask):
        """Returns token-level logits of shape (B, T, V)."""
        out = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        hidden = out["last_hidden_state"]
        logits = self.lm_head(hidden)
        return logits

def calculate_perplexity(logits, labels, attention_mask):
    """
    Compute per-example perplexity:
    - logits: (B*T, V)
    - labels & mask: (B*T)
    Returns perp of shape (B*T) then can be reshaped.
    """
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    shift_mask   = attention_mask[..., 1:].contiguous()

    loss_fn = nn.CrossEntropyLoss(reduction="none")
    loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)),
                   shift_labels.view(-1))
    loss = loss.view_as(shift_labels)
    loss = loss * shift_mask

    sum_loss = loss.sum(dim=1)
    token_count = shift_mask.sum(dim=1).clamp_min(1)
    mean_loss = sum_loss / token_count
    return torch.exp(mean_loss)

class MarginLoss(nn.Module):
    """Hinge-style margin loss between correct and incorrect PPLs."""
    def __init__(self, margin: float = 1.0):
        super().__init__()
        self.margin = margin

    def forward(self, correct_ppl, incorrect_ppls):
        # correct: (B,), incorrect: (B, C-1)
        correct = correct_ppl.unsqueeze(1).expand_as(incorrect_ppls)
        loss = F.relu(correct - incorrect_ppls + self.margin)
        return loss.mean()

class AnswerMarginDataset(Dataset):
    """Dataset for packing 5 candidate texts per example."""
    def __init__(self, df: pd.DataFrame, tokenizer: GPT2Tokenizer, max_length: int = 512):
        self.df = df.reset_index(drop=True)
        self.tok = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        prefix = row["prefix"]
        suffix = row["suffix"]
        choices = [row[f"choice_{i}"] for i in range(1,6)]
        example_texts = [f"{prefix}{c}{suffix}" for c in choices]
        label = int(row["answer"]) - 1
        return example_texts, label

    def collate_fn(self, batch):
        texts, labels = zip(*batch)
        flat_texts = sum(texts, [])
        enc = self.tok.batch_encode_plus(
            flat_texts,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        B = len(batch)
        input_ids = enc["input_ids"].view(B, 5, -1)
        attention_mask = enc["attention_mask"].view(B, 5, -1)
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(labels, dtype=torch.long)
        }

def load_data(path: str, split: str = None):
    """Load CSV and optionally filter by split column."""
    df = pd.read_csv(path, encoding="utf-8-sig")
    return df[df.split == split].reset_index(drop=True) if split else df

def save_model(model: nn.Module, optimizer, args, path: str):
    """Persist the model + optimizer state."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save({
        "model": model.state_dict(),
        "optim": optimizer.state_dict(),
        "args": args
    }, path)
    print(f"Saved model to {path}")

def evaluate(dataloader, model, lm_head_model, device):
    """Compute Top-1/2/3 accuracies and average PPL on a split."""
    model.eval()
    total = top1 = top2 = top3 = 0
    total_ppl = 0.0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", disable=TQDM_DISABLE):
            B, C, L = batch["input_ids"].shape
            ids = batch["input_ids"].view(B*C, L).to(device)
            mask = batch["attention_mask"].view(B*C, L).to(device)
            logits = model(ids, mask)
            ppl = calculate_perplexity(logits, ids, mask).view(B, C)

            total += B
            ranked = torch.argsort(ppl, dim=1)
            labels = batch["labels"].to(device)
            top1 += (ranked[:,0] == labels).sum().item()
            top2 += ((ranked[:,:2] == labels.unsqueeze(1)).any(1)).sum().item()
            top3 += ((ranked[:,:3] == labels.unsqueeze(1)).any(1)).sum().item()
            total_ppl += ppl.mean().item() * B

    return top1/total, top2/total, top3/total, total_ppl/total

def train(args):
    seed_everything(args.seed)
    device = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 1) 전체 CSV 로드 (split·grade 컬럼 포함)
    df = pd.read_csv(args.data_path, encoding="utf-8-sig")

    tokenizer = GPT2Tokenizer.from_pretrained(args.model_size)
    tokenizer.pad_token = tokenizer.eos_token
    model = AnswerMarginModel(args).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    loss_fn = MarginLoss(margin=args.margin)

    best_top1 = 0.0

    # 2) Curriculum: grade 2 ➔ grade 3
    for grade in [2, 3]:
        print(f"\n===== Training on grade {grade} =====")
        train_df = df[(df.split == "train") & (df.grade == grade)]
        dev_df   = df[(df.split == "valid") & (df.grade == grade)]
        if len(train_df) == 0:
            print(f"  (no train data for grade {grade}, skip)")
            continue

        train_ds = AnswerMarginDataset(train_df, tokenizer, args.max_length)
        dev_ds   = AnswerMarginDataset(dev_df, tokenizer, args.max_length)
        train_loader = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True,
                                  collate_fn=train_ds.collate_fn)
        dev_loader   = DataLoader(dev_ds, batch_size=args.batch_size, shuffle=False,
                                  collate_fn=dev_ds.collate_fn)

        # 3) 기존 epoch loop
        for epoch in range(1, args.epochs + 1):
            model.train()
            epoch_loss = 0.0
            for batch in tqdm(train_loader, desc=f"Grade {grade} Epoch {epoch}", disable=TQDM_DISABLE):
                optimizer.zero_grad()
                B, C, L = batch["input_ids"].shape
                ids  = batch["input_ids"].view(B*C, L).to(device)
                mask = batch["attention_mask"].view(B*C, L).to(device)
                logits = model(ids, mask)
                ppl = calculate_perplexity(logits, ids, mask).view(B, C)
                correct = ppl[torch.arange(B), batch["labels"].to(device)]
                wrongs  = torch.stack([ppl[i, torch.arange(C)!=batch["labels"][i]] for i in range(B)])
                loss = loss_fn(correct, wrongs)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            avg_loss = epoch_loss / len(train_loader)
            top1, top2, top3, avg_ppl = evaluate(dev_loader, model, None, device)
            print(f"[G{grade} E{epoch}] Loss: {avg_loss:.4f} | Dev Top1/2/3: {top1:.4f}/{top2:.4f}/{top3:.4f} | PPL: {avg_ppl:.4f}")

            if top1 > best_top1:
                best_top1 = top1
                save_model(model, optimizer, args, args.save_path)


def get_args():
    """Parse command-line arguments (ignores Jupyter args)."""
    p = argparse.ArgumentParser()
    p.add_argument("--data_path",    type=str, default="/content/drive/MyDrive/CSEG321/dataset/answer_margin.csv")
    p.add_argument("--model_size",   type=str, default="gpt2", choices=["gpt2","gpt2-medium","gpt2-large"])
    p.add_argument("--batch_size",   type=int, default=4)
    p.add_argument("--max_length",   type=int, default=512)
    p.add_argument("--lr",           type=float, default=5e-5)
    p.add_argument("--epochs",       type=int, default=50)
    p.add_argument("--margin",       type=float, default=1000.0)
    p.add_argument("--seed",         type=int, default=42)
    p.add_argument("--use_gpu",      action="store_true")
    # Jupyter friendly
    args = p.parse_args([])
    if torch.cuda.is_available(): args.use_gpu = True
    # set model dims
    if args.model_size == "gpt2":
        args.d, args.l, args.num_heads = 768,12,12
    elif args.model_size == "gpt2-medium":
        args.d, args.l, args.num_heads = 1024,24,16
    else:
        args.d, args.l, args.num_heads = 1280,36,20
    # save path
    args.save_path = f"/content/drive/MyDrive/CSEG321/models/answer_margin_{args.model_size}_{args.epochs}e_{args.lr}lr_curriculum.pt"
    return args

if __name__ == "__main__":
    args = get_args()
    train(args)

Using device: cuda

===== Training on grade 2 =====


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  5.73it/s]


[G2 E1] Loss: 6165411.5580 | Dev Top1/2/3: 0.0000/0.0000/0.6667 | PPL: 2117524.0000


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


[G2 E2] Loss: 21581.6561 | Dev Top1/2/3: 0.0000/0.0000/0.3333 | PPL: 432843.0312


...

[G3 E48] Loss: 649.1087 | Dev Top1/2/3: 0.2000/0.5333/0.8667 | PPL: 70184.9724


Evaluating: 100%|██████████| 4/4 [00:00<00:00,  4.76it/s]


[G3 E49] Loss: 683.1294 | Dev Top1/2/3: 0.2000/0.5333/0.8667 | PPL: 69676.2745


Evaluating: 100%|██████████| 4/4 [00:00<00:00,  4.78it/s]

[G3 E50] Loss: 564.1792 | Dev Top1/2/3: 0.2000/0.5333/0.8667 | PPL: 69234.3208


In [ ]:
"""
NLL-Ranking Trainer

Trains a GPT-2 model to pick the correct option by minimizing cross-entropy
on Negative-Log-Likelihood scores. Top-1/2/3 accuracy + average NLL.
"""

import argparse, os, random, sys, math
import numpy as np, pandas as pd, torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model     # custom GPT-2 implementation

TQDM_DISABLE = False  # set True in Colab if bar flickers

# --------------------------------------------------------------------- #
# 0. Utils
# --------------------------------------------------------------------- #
def seed_everything(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# --------------------------------------------------------------------- #
# 1. Dataset
# --------------------------------------------------------------------- #
class AnswerDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: GPT2Tokenizer, max_len=512):
        self.df, self.tok, self.max_len = df.reset_index(drop=True), tokenizer, max_len

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        choices = [row[f"choice_{i}"] for i in range(1, 6)]
        texts   = [f"{row.prefix}{c}{row.suffix}" for c in choices]
        label   = int(row.answer) - 1
        return texts, label

    def collate_fn(self, batch):
        texts, labels  = zip(*batch)
        flat = sum(texts, [])          # (B*5,) list
        enc  = self.tok(
            flat, padding="max_length", truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        B = len(batch)
        return {
            "input_ids":     enc["input_ids"].view(B, 5, -1),
            "attention_mask":enc["attention_mask"].view(B, 5, -1),
            "labels":        torch.tensor(labels, dtype=torch.long)
        }

# --------------------------------------------------------------------- #
# 2. Model: GPT-2 + LM-head to compute NLL
# --------------------------------------------------------------------- #
class NLLRankModel(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.gpt = GPT2Model.from_pretrained(
            model=args.model_size, d=args.d, l=args.l, num_heads=args.num_heads
        )
        vocab = GPT2Tokenizer.from_pretrained(args.model_size).vocab_size
        self.lm_head = nn.Linear(args.d, vocab, bias=False)
        for p in self.parameters(): p.requires_grad = True

    def forward(self, input_ids, attention_mask):
        """
        input_ids: (B, C, L)   attention_mask: (B, C, L)
        returns nll: (B, C)  -- lower is better
        """
        B, C, L = input_ids.shape
        flat_ids  = input_ids.view(B*C, L)
        flat_mask = attention_mask.view(B*C, L)

        h = self.gpt(flat_ids, attention_mask=flat_mask)["last_hidden_state"]
        logits = self.lm_head(h)                       # (B*C, L, V)

        # shift for LM loss
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = flat_ids[:, 1:].contiguous()
        shift_mask   = flat_mask[:, 1:].contiguous()

        loss_ = F.cross_entropy(
            shift_logits.view(-1, shift_logits.size(-1)),
            shift_labels.view(-1),
            reduction="none"
        ).view(flat_ids.size(0), -1) * shift_mask

        # token average NLL
        nll = loss_.sum(1) / shift_mask.sum(1).clamp_min(1)   # (B*C,)
        return nll.view(B, C)                                 # (B,5)

# --------------------------------------------------------------------- #
# 3. Train / Eval
# --------------------------------------------------------------------- #
def evaluate(loader, model, device):
    model.eval()
    tot = top1 = top2 = top3 = 0; tot_nll = 0.0
    with torch.no_grad():
        for batch in tqdm(loader, disable=TQDM_DISABLE, desc="Eval"):
            ids, mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
            nll = model(ids, mask)                 # (B, 5)
            scores = -nll                          # higher better
            rank = scores.argsort(dim=1, descending=True)
            y = batch["labels"].to(device)
            B = y.size(0)
            top1 += (rank[:, 0] == y).sum().item()
            top2 += ((rank[:, :2] == y.unsqueeze(1)).any(1)).sum().item()
            top3 += ((rank[:, :3] == y.unsqueeze(1)).any(1)).sum().item()
            tot  += B
            tot_nll += nll.mean().item()*B
    return top1/tot, top2/tot, top3/tot, tot_nll/tot

def train(args):
    seed_everything(args.seed)
    dev = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")
    print("Device:", dev)

    # load full CSV once
    df  = pd.read_csv(args.data_path, encoding="utf-8-sig")

    # prepare tokenizer (unchanged)
    tok = GPT2Tokenizer.from_pretrained(args.model_size)
    tok.pad_token = tok.eos_token

    # model & optimizer (unchanged)
    model = NLLRankModel(args).to(dev)
    optim = torch.optim.AdamW(model.parameters(), lr=args.lr)
    best = 0.0

    # ────────────────────────────────────────────────────
    #  Curriclum Learning: train on grade=2 then grade=3
    # ────────────────────────────────────────────────────
    for grade in [2, 3]:
        print(f"\n===== Curriculum Stage: Grade {grade} =====")
        # filter train/valid by grade
        trn = df[(df.split == "train") & (df.grade == grade)]
        val = df[(df.split == "valid") & (df.grade == grade)]
        if len(trn) == 0:
            print(f"No grade {grade} training data, skipping.")
            continue

        train_ds = AnswerDataset(trn, tok, args.max_length)
        val_ds   = AnswerDataset(val, tok, args.max_length)
        tr_loader= DataLoader(train_ds, batch_size=args.batch_size, shuffle=True,  collate_fn=train_ds.collate_fn)
        val_loader=DataLoader(val_ds,   batch_size=args.batch_size, shuffle=False, collate_fn=val_ds.collate_fn)

        # now run your existing epoch loop on this subset
        for ep in range(1, args.epochs+1):
            model.train(); ep_loss = 0.0
            for batch in tqdm(tr_loader, disable=TQDM_DISABLE, desc=f"Train G{grade} E{ep}"):
                optim.zero_grad()
                ids, mask = batch["input_ids"].to(dev), batch["attention_mask"].to(dev)
                nll = model(ids, mask)           # (B,5)
                loss = F.cross_entropy(-nll, batch["labels"].to(dev))
                loss.backward(); optim.step()
                ep_loss += loss.item()
            print(f"Grade {grade} Epoch {ep}  loss={ep_loss/len(tr_loader):.4f}")

            # evaluation (unchanged)
            t1,t2,t3,av_nll = evaluate(val_loader, model, dev)
            print(f"  Valid G{grade} Top1/2/3={t1:.3f}/{t2:.3f}/{t3:.3f}  AvgNLL={av_nll:.3f}")

            # checkpoint
            if t1 > best:
                best = t1
                save(model, optim, args)

def save(m, o, args):
    os.makedirs(os.path.dirname(args.save_path), exist_ok=True)
    torch.save({"model":m.state_dict(),"optim":o.state_dict(),"args":args}, args.save_path)
    print("Saved to", args.save_path)

def get_args():
    p=argparse.ArgumentParser();  # identical to original
    p.add_argument("--data_path", type=str, default="/content/drive/MyDrive/CSEG321/dataset/answer_margin.csv")
    p.add_argument("--model_size",type=str, default="gpt2", choices=["gpt2","gpt2-medium","gpt2-large"])
    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--max_length", type=int, default=512)
    p.add_argument("--lr",        type=float, default=5e-5)
    p.add_argument("--epochs",    type=int, default=50)
    p.add_argument("--seed",      type=int, default=42)
    p.add_argument("--use_gpu",   action="store_true")
    # Jupyter friendly
    args = p.parse_args([])
    if torch.cuda.is_available(): args.use_gpu = True
    if args.model_size=="gpt2":    args.d,args.l,args.num_heads=768,12,12
    elif args.model_size=="gpt2-medium": args.d,args.l,args.num_heads=1024,24,16
    else: args.d,args.l,args.num_heads=1280,36,20
    args.save_path=f"/content/drive/MyDrive/CSEG321/models/answer_nll_{args.model_size}_{args.epochs}e_curriculum.pt"
    return args

if __name__=="__main__":
    train(get_args())


Device: cuda

===== Curriculum Stage: Grade 2 =====


Train G2 E1: 100%|██████████| 7/7 [00:04<00:00,  1.56it/s]


Grade 2 Epoch 1  loss=1.6303


Eval: 100%|██████████| 1/1 [00:00<00:00,  5.73it/s]


  Valid G2 Top1/2/3=0.333/0.667/1.000  AvgNLL=19.350
Saved to /content/drive/MyDrive/CSEG321/models/answer_nll_gpt2_50e_curriculum.pt


...

Train G3 E15: 100%|██████████| 29/29 [00:18<00:00,  1.59it/s]


Grade 3 Epoch 15  loss=0.5527


Eval: 100%|██████████| 4/4 [00:00<00:00,  4.77it/s]


  Valid G3 Top1/2/3=0.467/0.467/0.600  AvgNLL=41.803


Train G3 E16:  93%|█████████▎| 27/29 [00:17<00:01,  1.59it/s]

In [13]:
"""
Dot-Product Rank-Head Trainer

Trains a GPT-2 encoder + linear rank head.
Top-1/2/3 accuracy + average softmax entropy on dev/test splits.
"""

import argparse, os, random, sys
import numpy as np, pandas as pd, torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model

TQDM_DISABLE=False

# --------------------------------------------------------------------- #
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
# --------------------------------------------------------------------- #
class AnswerDataset(Dataset):
    def __init__(self, df, tok, max_len=512):
        self.df, self.tok, self.max_len = df.reset_index(drop=True), tok, max_len
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r=self.df.iloc[idx]
        choices=[r[f"choice_{i}"] for i in range(1,6)]
        texts=[f"{r.prefix}{c}{r.suffix}" for c in choices]
        lab=int(r.answer)-1
        return texts, lab
    def collate_fn(self,b):
        txts, labs=zip(*b); flat=sum(txts,[])
        enc=self.tok(flat,padding="max_length",truncation=True,
                     max_length=self.max_len,return_tensors="pt")
        B=len(b)
        return {"input_ids":enc.input_ids.view(B,5,-1),
                "attention_mask":enc.attention_mask.view(B,5,-1),
                "labels":torch.tensor(labs)}
# --------------------------------------------------------------------- #
class DotRankModel(nn.Module):
    def __init__(self,args):
        super().__init__()
        self.enc=GPT2Model.from_pretrained(model=args.model_size,
                                           d=args.d,l=args.l,num_heads=args.num_heads)
        self.head=nn.Linear(args.d,1)
        for p in self.parameters(): p.requires_grad=True
    def forward(self,ids,mask):
        B,C,L=ids.shape
        flat_ids,flat_mask=ids.view(B*C,L),mask.view(B*C,L)
        h=self.enc(flat_ids,attention_mask=flat_mask)["last_hidden_state"]
        last_idx=flat_mask.sum(1)-1
        pooled=h[torch.arange(h.size(0)), last_idx]  # (B*C,d)
        score=self.head(pooled).view(B,C)            # (B,5)
        return score
# --------------------------------------------------------------------- #
def evaluate(loader,model,dev):
    model.eval(); tot=top1=top2=top3=0; tot_ent=0
    with torch.no_grad():
        for bt in tqdm(loader,disable=TQDM_DISABLE,desc="Eval"):
            s=model(bt["input_ids"].to(dev),bt["attention_mask"].to(dev))
            rank=s.argsort(dim=1,descending=True)
            y=bt["labels"].to(dev); B=y.size(0)
            top1+=(rank[:,0]==y).sum().item()
            top2+=((rank[:,:2]==y.unsqueeze(1)).any(1)).sum().item()
            top3+=((rank[:,:3]==y.unsqueeze(1)).any(1)).sum().item()
            tot+=B
            probs=F.softmax(s,dim=1)
            ent=-(probs*probs.log()).sum(1).mean().item()
            tot_ent+=ent*B
    return top1/tot,top2/tot,top3/tot,tot_ent/tot
# --------------------------------------------------------------------- #
def train(args):
    seed_everything(args.seed)
    device = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # 1) CSV 한 번만 로드
    df = pd.read_csv(args.data_path, encoding="utf-8-sig")

    # 2) 토크나이저 & model/optimizer 초기화
    tok = GPT2Tokenizer.from_pretrained(args.model_size)
    tok.pad_token = tok.eos_token
    model = DotRankModel(args).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr)

    best_top1 = 0.0

    # 3) Curriculum: grade 2 → grade 3 순차 학습
    for grade in [2, 3]:
        print(f"\n=== Curriculum Stage: Grade {grade} ===")
        train_df = df[(df.split == "train") & (df.grade == grade)]
        valid_df = df[(df.split == "valid") & (df.grade == grade)]

        if len(train_df) == 0:
            print(f"No train data for grade {grade}, skipping.")
            continue

        # 4) DataLoader 생성
        tr_ds = AnswerDataset(train_df, tok, args.max_length)
        va_ds = AnswerDataset(valid_df, tok, args.max_length)
        tr_ld = DataLoader(tr_ds, batch_size=args.batch_size, shuffle=True, collate_fn=tr_ds.collate_fn)
        va_ld = DataLoader(va_ds, batch_size=args.batch_size, shuffle=False, collate_fn=va_ds.collate_fn)

        # 5) 기존 epoch 루프 재사용
        for ep in range(1, args.epochs+1):
            model.train()
            ep_loss = 0.0
            for bt in tqdm(tr_ld, desc=f"[G{grade}] Train Ep{ep}", disable=TQDM_DISABLE):
                opt.zero_grad()
                s = model(bt["input_ids"].to(device), bt["attention_mask"].to(device))
                loss = F.cross_entropy(s, bt["labels"].to(device))
                loss.backward()
                opt.step()
                ep_loss += loss.item()
            print(f"[G{grade}] Ep{ep} train_loss = {ep_loss/len(tr_ld):.4f}")

            # 6) Validation
            t1, t2, t3, ent = evaluate(va_ld, model, device)
            print(f"[G{grade}] Ep{ep} valid Top1/2/3 = {t1:.3f}/{t2:.3f}/{t3:.3f} | Ent={ent:.3f}")

            # 7) Checkpoint
            if t1 > best_top1:
                best_top1 = t1
                save(model, opt, args)

def save(m,o,args):
    os.makedirs(os.path.dirname(args.save_path),exist_ok=True)
    torch.save({"model":m.state_dict(),"optim":o.state_dict(),"args":args},args.save_path)
    print("Saved to",args.save_path)
def get_args():
    p=argparse.ArgumentParser()
    p.add_argument("--data_path",type=str,default="/content/drive/MyDrive/CSEG321/dataset/answer_margin.csv")
    p.add_argument("--model_size",type=str,default="gpt2",choices=["gpt2","gpt2-medium","gpt2-large"])
    p.add_argument("--batch_size",type=int,default=4)
    p.add_argument("--max_length",type=int,default=512)
    p.add_argument("--lr",type=float,default=5e-5)
    p.add_argument("--epochs",type=int,default=50)
    p.add_argument("--seed",type=int,default=42)
    p.add_argument("--use_gpu",action="store_true")
    # Jupyter friendly
    args = p.parse_args([])
    if torch.cuda.is_available(): args.use_gpu = True
    if args.model_size=="gpt2": args.d,args.l,args.num_heads=768,12,12
    elif args.model_size=="gpt2-medium": args.d,args.l,args.num_heads=1024,24,16
    else: args.d,args.l,args.num_heads=1280,36,20
    args.save_path=f"/content/drive/MyDrive/CSEG321/models/answer_dot_{args.model_size}_{args.epochs}e_curriculum.pt"
    return args
if __name__=="__main__": train(get_args())


Using device: cuda

=== Curriculum Stage: Grade 2 ===


[G2] Train Ep1: 100%|██████████| 7/7 [00:03<00:00,  2.07it/s]


[G2] Ep1 train_loss = 1.8211


Eval: 100%|██████████| 1/1 [00:00<00:00,  7.29it/s]


[G2] Ep1 valid Top1/2/3 = 0.333/0.333/0.333 | Ent=1.601
Saved to /content/drive/MyDrive/CSEG321/models/answer_dot_gpt2_50e_curriculum.pt


...

[G3] Ep49 valid Top1/2/3 = 0.133/0.667/0.867 | Ent=1.128


[G3] Train Ep50: 100%|██████████| 29/29 [00:13<00:00,  2.12it/s]


[G3] Ep50 train_loss = 0.0323


Eval: 100%|██████████| 4/4 [00:00<00:00,  6.16it/s]

[G3] Ep50 valid Top1/2/3 = 0.267/0.667/0.933 | Ent=1.106
